In [28]:
import pandas as pd
sber_data = pd.read_csv('data/sber_data.csv')
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(sber_data.isnull().count(),sber_data.isnull().median(),sber_data.isnull().sum())

In [ ]:
fig = plt.figure(figsize=(10, 5))
sns.boxplot(data=sber_data, x='price_doc', y='ecology');

In [ ]:
jointplot = sns.jointplot(
    data=sber_data, 
    x='kremlin_km', 
    y='price_doc',
    height=7,
)

In [ ]:
#МЕТОД МЕЖКВАРТИЛЬНОГО РАЗМАХА (МЕТОД ТЬЮКИ)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 4))
histplot = sns.histplot(data=sber_data, x='price_doc', ax=axes[0]);
histplot.set_title('Full Square Distribution');
boxplot = sns.boxplot(data=sber_data, x='price_doc', ax=axes[1]);
boxplot.set_title('Full Square Boxplot');

In [ ]:
# метод межквартильного размаха
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_iqr_mod(sber_data, 'price_doc', left=3, right=3, log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [ ]:
# МЕТОД Z-ОТКЛОНЕНИЙ (МЕТОД СИГМ)
def outliers_z_score_mod(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score_mod(sber_data, 'price_doc', log_scale=True, left=3, right=3)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

In [41]:
# FINAL TASK
import pandas as pd
diabetes_data = pd.read_csv('data/diabetes_data.csv')

In [42]:
dupl_columns = list(diabetes_data.columns)
diabetes_data_dedupped = diabetes_data.drop_duplicates(subset=dupl_columns)

In [43]:
#список неинформативных признаков
low_information_cols = [] 
#цикл по всем столбцам
for col in diabetes_data_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_data_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_data_dedupped[col].nunique() / diabetes_data_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
diabetes_data_DNI = diabetes_data_dedupped.drop(low_information_cols, axis=1)

Gender: 100.0% одинаковых значений


In [ ]:
import numpy as np
for col in diabetes_data_DNI.columns:
    diabetes_data_DNI[col] = diabetes_data_DNI[col].apply(lambda x: np.nan if x==0 else x)
diabetes_data_DNI.isnull().mean()

In [45]:
thresh = diabetes_data_DNI.shape[0]*0.7
diabetes_data_DNI = diabetes_data_DNI.dropna(thresh=thresh, axis=1)
print(diabetes_data_DNI.shape[1])

7


In [47]:
m = diabetes_data_DNI.shape[1]
diabetes_data_DNI = diabetes_data_DNI.dropna(thresh=m-2, axis=0)
print(diabetes_data_DNI.shape[0])

752


In [48]:
null_data = diabetes_data_DNI.isnull().sum()
cols = null_data[null_data>0].index
for col in cols:
    diabetes_data_DNI[col] = diabetes_data_DNI[col].fillna(diabetes_data_DNI[col].median())
print(diabetes_data_DNI['SkinThickness'].mean().round(1))

29.1


In [49]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes_data_DNI, 'SkinThickness')
print(outliers.shape[0])

81


In [50]:
def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_z_score_mod(diabetes_data_DNI, 'SkinThickness')
print(outliers.shape[0])

4


In [51]:
def outliers_iqr_mod(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, _ = outliers_iqr_mod(diabetes_data_DNI, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr_mod(diabetes_data_DNI, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0] - outliers_log.shape[0])

29
